In [64]:
import requests
import pandas as pd

# Tarifas das Distribuidoras

Dados de: https://dadosabertos.aneel.gov.br/dataset/tarifas-distribuidoras-energia-eletrica/resource/fcf2906c-7c32-4b9b-a637-054e7a5234f4

In [101]:
def get_tarifas_distribuidoras(CNPJ):
    final_df = pd.DataFrame()
    base_url = "https://dadosabertos.aneel.gov.br"
    url = f"{base_url}/api/3/action/datastore_search?resource_id=fcf2906c-7c32-4b9b-a637-054e7a5234f4&q={CNPJ}"
    
    while url:
        data = requests.get(url).json()
        df = pd.json_normalize(data["result"]["records"])
        
        if len(df) > 0:
            final_df = pd.concat([final_df, df], axis=0)
            url = base_url + data["result"]["_links"]["next"]
        else:
            url = None

    return final_df

In [115]:
def trata_df_tarifas(df):
    df["DatInicioVigencia"] = pd.to_datetime(df["DatInicioVigencia"])
    filter_DscSubGrupo = df["DscSubGrupo"] == "B1"
    filter_DscModalidadeTarifaria = df["DscModalidadeTarifaria"] == "Convencional"
    filter_DscSubClasse = df["DscSubClasse"] == "Residencial"
    filter_DscBaseTarifaria = df["DscBaseTarifaria"] == "Base Econômica"
    filter_DatInicioVigencia = df["DatInicioVigencia"] == df["DatInicioVigencia"].max()
    return df[filter_DscSubGrupo & filter_DscModalidadeTarifaria & filter_DscSubClasse & filter_DscBaseTarifaria & filter_DatInicioVigencia]

In [136]:
empresas = {
    "Light": "60444437000146",
    "Enel RJ": "33050071000158"
}

dataframes = []
for empresa in empresas.keys():
    dataframes.append(get_tarifas_distribuidoras(empresas[empresa]).pipe(trata_df_tarifas))

df_tarifas = pd.concat(dataframes)

In [260]:
df_tarifas['VlrTUSD'] = df_tarifas['VlrTUSD'].str.replace(',', '.').astype(float)
df_tarifas['VlrTE'] = df_tarifas['VlrTE'].str.replace(',', '.').astype(float)

In [261]:
df_tarifas

,_id,DatGeracaoConjuntoDados,DscREH,SigAgente,NumCNPJDistribuidora,DatInicioVigencia,DatFimVigencia,DscBaseTarifaria,DscSubGrupo,DscModalidadeTarifaria,DscClasse,DscSubClasse,DscDetalhe,NomPostoTarifario,DscUnidadeTerciaria,SigAgenteAcessante,VlrTUSD,VlrTE,rank
63,67736,2022-06-01,"RESOLUÇÃO HOMOLOGATÓRIA Nº 3.014, DE 15 DE MAR...",LIGHT,60444437000146,2022-03-15,2023-03-14,Base Econômica,B1,Convencional,Residencial,Residencial,Não se aplica,Não se aplica,R$/MWh,Não se aplica,463.33,346.57,0.057309
42,70927,2022-06-01,"RESOLUÇÃO HOMOLOGATÓRIA Nº 3.015, DE 15 DE MAR...",ENEL RJ,33050071000158,2022-03-15,2023-03-14,Base Econômica,B1,Convencional,Residencial,Residencial,Não se aplica,Não se aplica,R$/MWh,Não se aplica,523.52,307.44,0.057309


In [295]:
df_tarifas.to_csv("../dados/TarifasDistribuidoras.csv", index=False)

# Vendas no Leilão

Dados de: https://www.ccee.org.br/web/guest/mercado/leilao-mercado

In [263]:
dados_vendedores = pd.read_csv("../dados/ConsolidadoVendedoresLeilao.csv",sep=';',encoding='mbcs')

In [264]:
dados_vendedores.columns

Index(['ID Resultado Vendedor', 'Ano do Leilão', 'Tipo Leilão', 'Leilão',
       'Denominação do Leilão', 'Edital do Leilão ANEEL', 'Sigla Vendedor',
       'CNPJ', 'Produto', 'CEG', 'Nome da Usina', 'Situação', 'UF', 'Fonte',
       'Rio/Combustível', 'Submercado', 'Investimento na Data do Leilão (R$)',
       'Investimento Atualizado (R$.mi)', 'Potência (MW)',
       'Potência Injetada (MW)', 'Disponibilidade de Potência (MW)',
       'Potência C.C (MWp)', 'Garantia Física (MW médio)',
       'Preço de Venda na Data do Leilão (R$/MWh)', 'Unidade de Preço',
       'ICE (R$/MWh)', 'CVU (R$/MWh)', 'Montante Financeiro Data Leilão (R$)',
       'Preço de Venda Atualizado (R$/MWh)',
       'Montante Financeiro Atualizado (R$)', 'Início Suprimento',
       'Fim Suprimento', 'Entrega Escalonada', 'IPCA na Data do leilão',
       'IPCA Abr/2022'],
      dtype='object')

In [265]:
dados_vendedores["Denominação do Leilão"].unique()

array(['Leilão de Energia Existente', 'Leilão de Energia Nova',
       'Procedimento Competitivo Simplificado',
       'Leilão de Reserva de Capacidade'], dtype=object)

In [266]:
dados_vendedores = dados_vendedores[dados_vendedores["Denominação do Leilão"].isin(['Leilão de Energia Existente', 'Leilão de Energia Nova'])]

# Sustentabilidade

Dados Criados Manualmente através de Pesquisa sobre Geradores e suas Fontes. Provavelmente existem índices que podem ser utilizados mas não encontramos por enquanto.

In [297]:
df_sustentabilidade = dados_vendedores[["CNPJ", "Sigla Vendedor", "Fonte"]].drop_duplicates().sort_values("CNPJ")

In [298]:
df_sustentabilidade["NotaSustentabilidade"] = df_sustentabilidade["Fonte"].replace({
    "Eólica": 9,
    'Solar Fotovoltaica': 9, 
    'Hidrelétrica (PCH)': 8,
    'Biomassa': 8, 
    'Hidrelétrica (UHE)': 8,
    'Resíduo Sólido Urbano': 8, 
    'Gás Natural': 6, 
    'Óleo Combustível': 6,
    '-': 5
})

In [299]:
df_sustentabilidade

,CNPJ,Sigla Vendedor,Fonte,NotaSustentabilidade
114,00.357.038/0001-16,ELETRONORTE,-,5
55,00.622.416/0001-41,STATKRAFT,Eólica,9
91,07.674.341/0001-91,UTE Uberaba 2,Biomassa,8
51,07.912.062/0001-19,LAGUNA,Biomassa,8
90,07.912.062/0001-19,UTE LAGUNA,Biomassa,8
52,07.966.116/0001-29,ALBIOMA,Biomassa,8
4,08.147.388/0001-60,CACH 5 VEADOS A4,Hidrelétrica (PCH),8
3,08.147.432/0001-31,RINCAO S MIGUEL A4,Hidrelétrica (PCH),8
2,08.323.274/0001-23,COPREL,Hidrelétrica (PCH),8
110,08.364.948/0001-38,ALUPAR,-,5


In [300]:
def agrupa_fontes(df):
    return pd.DataFrame({
        "CNPJ": [df.iloc[0]["CNPJ"]],
        "Sigla Vendedor": [df.iloc[0]["Sigla Vendedor"]],
        "Fonte": [df.iloc[0]["Fonte"]],
        "NotaSustentabilidade": [df["NotaSustentabilidade"].mean()]
    })

df_sustentabilidade = df_sustentabilidade.groupby("CNPJ", as_index=False).apply(agrupa_fontes).droplevel(1)

In [301]:
df_sustentabilidade.head(5)

,CNPJ,Sigla Vendedor,Fonte,NotaSustentabilidade
0,00.357.038/0001-16,ELETRONORTE,-,5.0
1,00.622.416/0001-41,STATKRAFT,Eólica,9.0
2,07.674.341/0001-91,UTE Uberaba 2,Biomassa,8.0
3,07.912.062/0001-19,LAGUNA,Biomassa,8.0
4,07.966.116/0001-29,ALBIOMA,Biomassa,8.0


In [272]:
df_sustentabilidade.to_csv("../dados/InfoGeradoras.csv", index=False)

# Precos de Vendas

In [276]:
dados_venda = dados_vendedores[[
    "Sigla Vendedor", 
    "CNPJ", 
    "Preço de Venda Atualizado (R$/MWh)", 
    'Unidade de Preço', 
    'UF',
    'Início Suprimento',
    'Ano do Leilão'
]].rename(columns={
    "Sigla Vendedor": "SiglaVendedor",
    "Preço de Venda Atualizado (R$/MWh)": "PrecoVenda",
    'Unidade de Preço': "UnidadePreco",
    "Ano do Leilão": "AnoLeilao",
    'Início Suprimento': "InicioSuprimento"
})

In [278]:
dados_venda["InicioSuprimento"] = pd.to_datetime(dados_venda["InicioSuprimento"])
dados_venda['PrecoVenda'] = dados_venda['PrecoVenda'].str.replace(',', '.').astype(float)

In [279]:
dados_venda = dados_venda.sort_values("InicioSuprimento", ascending=True)

In [280]:
dados_venda = dados_venda[dados_venda["CNPJ"].isin(df_sustentabilidade["CNPJ"])]

In [281]:
dados_venda.head()

,SiglaVendedor,CNPJ,PrecoVenda,UnidadePreco,UF,InicioSuprimento,AnoLeilao
110,ALUPAR,08.364.948/0001-38,222.71,R$/MWh,-,2022-01-01,2021
111,BRASIL COM,13.145.928/0001-06,221.65,R$/MWh,-,2022-01-01,2021
112,COPEL COM,19.125.927/0001-86,218.26,R$/MWh,-,2022-01-01,2021
113,SAFIRA COM,09.495.582/0001-07,222.71,R$/MWh,-,2022-01-01,2021
118,VIVAZ ENERGIA,26.537.119/0001-91,212.09,R$/MWh,-,2023-01-01,2021


In [285]:
def agrupa_fontes(df):
    return pd.DataFrame({
        "CNPJ": [df.iloc[0]["CNPJ"]],
        "SiglaVendedor": [df.iloc[0]["SiglaVendedor"]],
        "PrecoVenda": [df["PrecoVenda"].mean()],
        "UnidadePreco": [df.iloc[0]["UnidadePreco"]],
        "UF": [df.iloc[0]["UF"]],
        "InicioSuprimento": [df["InicioSuprimento"].min()],
        "AnoLeilao": [df.iloc[0]["AnoLeilao"]],
    })

dados_venda = dados_venda.groupby("CNPJ", as_index=False).apply(agrupa_fontes).droplevel(1)

In [286]:
dados_venda.head()

,CNPJ,SiglaVendedor,PrecoVenda,UnidadePreco,UF,InicioSuprimento,AnoLeilao
0,00.357.038/0001-16,ELETRONORTE,212.11,R$/MWh,-,2023-01-01,2021
1,00.622.416/0001-41,STATKRAFT,146.61,R$/MWh,BA,2026-01-01,2021
2,07.674.341/0001-91,UTE Uberaba 2,296.55,R$/MWh,MG,2026-01-01,2021
3,07.912.062/0001-19,LAGUNA,256.00,R$/MWh,MS,2025-01-01,2021
4,07.966.116/0001-29,ALBIOMA,216.97,R$/MWh,GO,2025-01-01,2021


In [294]:
dados_venda.to_csv("../dados/PrecoLeiloes.csv", index=False)